<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Plot-of-Surface-Pressure-on-MPAS-grid" data-toc-modified-id="Plot-of-Surface-Pressure-on-MPAS-grid-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plot of Surface Pressure on MPAS grid</a></span><ul class="toc-item"><li><span><a href="#Get-Data" data-toc-modified-id="Get-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get Data</a></span></li><li><span><a href="#Read-Data-from-MPAS-Grid" data-toc-modified-id="Read-Data-from-MPAS-Grid-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Read Data from MPAS Grid</a></span></li><li><span><a href="#Convert-to-degrees-from-radians" data-toc-modified-id="Convert-to-degrees-from-radians-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Convert to degrees from radians</a></span></li><li><span><a href="#Global-tripcolor-plot" data-toc-modified-id="Global-tripcolor-plot-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Global tripcolor plot</a></span></li></ul></li></ul></div>

# Plot of Surface Pressure on MPAS grid

In [ ]:
%matplotlib inline
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.tri as tri
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

## Get Data

In [ ]:
from ngallery_utils import DATASETS
DATASETS.registry_files

In [ ]:
path = DATASETS.fetch("MPAS.nc")

## Read Data from MPAS Grid

In [ ]:
ds = xr.open_dataset(path)
ds = ds.squeeze()
ds

In [ ]:
# Not sure what the pressure units are, there's not much metadata info on this file
ds["surface_pressure"] = ds["surface_pressure"] / 1000.

## Convert to degrees from radians

In [ ]:
ds["lonCell"] = np.degrees(ds.lonCell)
ds["latCell"] = np.degrees(ds.latCell)

## Global tripcolor plot

In [ ]:
# Add cyclic point before computing the triangulation
ds["lonCell"] = ((ds["lonCell"] + 180) % 360) - 180
triang = tri.Triangulation(ds.lonCell, ds.latCell)

In [ ]:
import cmaps as cm  # for NCL colormaps

In [ ]:
# Don't use white by cutting out the first section of the colormap
from matplotlib.colors import LinearSegmentedColormap
interval = np.hstack([np.linspace(0.15, 1)])
colors = cm.WhiteBlueGreenYellowRed(interval)
c = LinearSegmentedColormap.from_list('name', colors)

In [ ]:
%%time

fig = plt.figure(figsize=(13, 13))
ax = plt.subplot(111, projection=ccrs.PlateCarree())
mm = ax.tripcolor(triang,
                  ds.surface_pressure,
                  edgecolors='k',
                  lw=0.01,
                  alpha=1,
                  transform=ccrs.PlateCarree(),
                  cmap=c,
                  vmin=55.0,
                  vmax=100.)
ax.set_global()
ax.coastlines(lw=1.5, resolution='110m')
gl = ax.gridlines(draw_labels=True)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabels_top = gl.ylabels_right = False
gl.xlines = gl.ylines = False
plt.colorbar(mm, orientation='horizontal', pad=0.03)
plt.title(f"Surface pressure on MPAS grid ({len(ds.lonCell)} cells)",
          fontweight="bold",
          fontsize=14)

plt.show()


-------------

**NCL Script**: https://www.ncl.ucar.edu/Applications/Scripts/mpas_1.ncl

**NCL Plot**:

![](https://www.ncl.ucar.edu/Applications/Images/mpas_1_lg.png)

In [ ]:
%load_ext watermark
%watermark -ud -iv -m -g -h